# Competition: Sberbank Russian Housing Market

Group Members:
* Bayan Ali
* Majid Sohrabi

Group: ISSA (Data Science)

# Data Preprocessing

In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import datetime

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#load data
train = pd.read_csv('/content/drive/MyDrive/MLDM-Competition-HSE/Data/train.csv', parse_dates=['timestamp'])
test = pd.read_csv('/content/drive/MyDrive/MLDM-Competition-HSE/Data/test.csv', parse_dates=['timestamp'])

In [4]:
train[:5]

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,...,cafe_sum_3000_min_price_avg,cafe_sum_3000_max_price_avg,cafe_avg_price_3000,cafe_count_3000_na_price,cafe_count_3000_price_500,cafe_count_3000_price_1000,cafe_count_3000_price_1500,cafe_count_3000_price_2500,cafe_count_3000_price_4000,cafe_count_3000_price_high,big_church_count_3000,church_count_3000,mosque_count_3000,leisure_count_3000,sport_count_3000,market_count_3000,green_part_5000,prom_part_5000,office_count_5000,office_sqm_5000,trc_count_5000,trc_sqm_5000,cafe_count_5000,cafe_sum_5000_min_price_avg,cafe_sum_5000_max_price_avg,cafe_avg_price_5000,cafe_count_5000_na_price,cafe_count_5000_price_500,cafe_count_5000_price_1000,cafe_count_5000_price_1500,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,1,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Bibirevo,6.407578e+06,155572,0.189727,0.000070,9576,5001.0,5,10309,11065.0,5,0,240.0,1,0,7,3,no,0,16,1,no,no,no,no,no,no,no,...,639.68,1079.37,859.52,5,21,22,16,3,1,0,2,4,0,0,21,1,13.09,13.31,29,807385,52,4036616,152,708.57,1185.71,947.14,12,39,48,40,9,4,0,13,22,1,0,52,4,5850000
1,2,2011-08-23,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Nagatinskij Zaton,9.589337e+06,115352,0.372602,0.049637,6880,3119.0,5,7759,6237.0,8,0,229.0,1,0,6,1,yes,1,3,0,no,no,no,no,no,no,no,...,631.03,1086.21,858.62,1,11,11,4,2,1,0,1,7,0,6,19,1,10.26,27.47,66,2690465,40,2034942,177,673.81,1148.81,911.31,9,49,65,36,15,3,0,15,29,1,10,66,14,6000000
2,3,2011-08-27,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Tekstil'shhiki,4.808270e+06,101708,0.112560,0.118537,5879,1463.0,4,6207,5580.0,7,0,1183.0,1,0,5,1,no,0,0,1,no,no,no,yes,no,no,no,...,697.44,1192.31,944.87,2,9,17,9,3,1,0,0,11,0,0,20,6,13.69,21.58,43,1478160,35,1572990,122,702.68,1196.43,949.55,10,29,45,25,10,3,0,11,27,0,4,67,10,5700000
3,4,2011-09-01,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Mitino,1.258354e+07,178473,0.194703,0.069753,13087,6839.0,9,13670,17063.0,10,0,NaN,1,0,17,6,no,0,11,4,no,no,no,no,no,no,no,...,718.75,1218.75,968.75,0,5,14,10,3,0,0,1,2,0,0,18,3,14.18,3.89,8,244166,22,942180,61,931.58,1552.63,1242.11,4,7,21,15,11,2,1,4,4,0,0,26,3,13100000
4,5,2011-09-05,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Basmannoe,8.398461e+06,108171,0.015234,0.037316,5706,3240.0,7,6748,7770.0,9,0,562.0,4,2,25,2,no,0,10,93,no,no,no,yes,yes,no,no,...,853.03,1410.45,1131.74,63,266,267,262,149,57,4,70,121,1,40,77,5,8.38,10.92,689,8404624,114,3503058,2283,853.88,1411.45,1132.66,143,566,578,552,319,108,17,135,236,2,91,195,14,16331452


In [5]:
test[:5]

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,...,cafe_count_3000,cafe_sum_3000_min_price_avg,cafe_sum_3000_max_price_avg,cafe_avg_price_3000,cafe_count_3000_na_price,cafe_count_3000_price_500,cafe_count_3000_price_1000,cafe_count_3000_price_1500,cafe_count_3000_price_2500,cafe_count_3000_price_4000,cafe_count_3000_price_high,big_church_count_3000,church_count_3000,mosque_count_3000,leisure_count_3000,sport_count_3000,market_count_3000,green_part_5000,prom_part_5000,office_count_5000,office_sqm_5000,trc_count_5000,trc_sqm_5000,cafe_count_5000,cafe_sum_5000_min_price_avg,cafe_sum_5000_max_price_avg,cafe_avg_price_5000,cafe_count_5000_na_price,cafe_count_5000_price_500,cafe_count_5000_price_1000,cafe_count_5000_price_1500,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000
0,30474,2015-07-01,39.0,20.7,2,9,1,1998.0,1,8.9,3.0,Investment,Juzhnoe Butovo,2.615514e+07,178264,0.137846,0.041116,14080,11926.0,11,14892,24750.0,13,1,NaN,1,0,13,4,no,0,4,4,no,no,no,no,no,no,no,...,12,781.82,1227.27,1004.55,1,2,2,7,0,0,0,1,3,1,0,7,0,21.58,4.69,1,37550,8,299166,19,676.47,1088.24,882.35,2,5,4,8,0,0,0,1,10,1,0,14,1
1,30475,2015-07-01,79.2,NaN,8,17,1,0.0,3,1.0,1.0,OwnerOccupier,Poselenie Vnukovskoe,2.553630e+07,4001,0.496315,0.007122,275,NaN,0,264,NaN,0,0,NaN,0,0,0,0,no,0,1,0,no,no,no,no,no,no,no,...,10,680.00,1200.00,940.00,0,1,6,2,1,0,0,1,5,0,0,7,0,39.10,7.70,2,177300,6,231300,20,733.33,1250.00,991.67,2,4,8,4,1,1,0,2,11,0,1,12,1
2,30476,2015-07-01,40.5,25.1,3,5,2,1960.0,2,4.8,2.0,Investment,Perovo,9.946335e+06,139322,0.065409,0.225825,6400,2232.0,7,6558,7966.0,7,0,1548.0,3,0,13,0,no,0,2,7,yes,no,yes,yes,no,no,no,...,37,608.33,1069.44,838.89,1,8,19,7,2,0,0,2,3,0,5,22,4,25.62,13.59,27,427889,26,1024431,179,668.97,1132.18,900.57,5,53,64,42,11,4,0,10,21,0,10,71,11
3,30477,2015-07-01,62.8,36.0,17,17,1,2016.0,2,62.8,3.0,OwnerOccupier,Poselenie Voskresenskoe,2.149409e+07,7122,0.262459,0.017647,489,NaN,0,469,NaN,0,0,NaN,0,0,0,2,no,0,0,0,no,no,no,no,no,no,no,...,1,300.00,500.00,400.00,0,1,0,0,0,0,0,0,4,0,0,0,0,24.25,1.66,0,0,0,0,5,1560.00,2500.00,2030.00,0,1,0,1,1,2,0,0,10,0,0,2,0
4,30478,2015-07-01,40.0,40.0,17,17,1,0.0,1,1.0,1.0,OwnerOccupier,Poselenie Vnukovskoe,2.553630e+07,4001,0.496315,0.007122,275,NaN,0,264,NaN,0,0,NaN,0,0,0,0,no,0,1,0,no,no,no,no,no,no,no,...,9,700.00,1222.22,961.11,0,1,5,2,1,0,0,1,4,0,0,6,0,35.62,6.96,1,117300,4,201300,20,747.37,1263.16,1005.26,1,4,8,5,1,1,0,2,12,0,1,11,1


In [6]:
id_test = test.id
id_train = train.id

In [7]:
#preprocessing Data / cleaning Data
# Train Data
# 1.full_sq: total area in square meters, including loggias, balconies and other non-residential areas
# life_sq: living area in square meters, excluding loggias, balconies and other non-residential areas
# so life_sq <= full_sq and if life_sq > full_sq we will put NaN 
index = train[train.life_sq > train.full_sq].index
train.iloc[index]["life_sq"] = np.NaN
# 2. Put NaN when life_sq < 5
index = train[train.life_sq < 5].index
train.iloc[index]["life_sq"] = np.NaN
# 3. Put NaN when full_sq < 5
index = train[train.full_sq < 5].index
train.iloc[index]["full_sq"] = np.NaN
# 4. train.iloc[13117]["kitch_sq"] = 1970.0  # the value should be for build_year
train.iloc[13117]["build_year"] = train.iloc[13117]["kitch_sq"]
# 5. kitch_sq should be < life_sq so if kitch_sq >= life_sq ==> NaN
index = train[train.kitch_sq >= train.life_sq].index
train.iloc[index]["kitch_sq"] = np.NaN
# 6. if  num_room = 0 ==> num_room = NaN
index = train[train.num_room == 0].index 
train.iloc[index]["num_room"] = np.NaN
# 7. replace with NaN if num romm >= 6 
#index = [10076, 11621, 17764, 19390, 24007, 26713, 29172]
index = train.iloc[(train.num_room >= 6).values].index
train.iloc[index]["num_room"] = np.NaN
# 8. 
index = train[train.floor == 0].index
train.iloc[index]["floor"] = np.NaN
index = train[train.max_floor == 0].index
train.iloc[index]["max_floor"] = np.NaN
# 9. floor should be <= max floor
index = train[train.floor > train.max_floor].index
train.iloc[index]["max_floor"] = np.NaN
# 10. there are just 4 conditions
index = train.iloc[(train.state >= 5).values].index
train.iloc[index]["state"] = np.NaN


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [8]:
# Test Data
# 1. Same in pint 1 for train data.
index = test[test.life_sq > test.full_sq].index
test.iloc[index]["life_sq"] = np.NaN
# 2. life_sq for these index [601,1896,2791] equal full_sq but need (.)
index = [601,1896,2791]
test.iloc[index]["life_sq"] = test.iloc[index]["full_sq"]
# 3. Put NaN when life_sq < 5
index = test[test.life_sq < 5].index
test.iloc[index]["life_sq"] = np.NaN
# 4. Put NaN when full_sq < 5
index = test[test.full_sq < 5].index
test.iloc[index]["full_sq"] = np.NaN
# 5. kitch_sq should be < life_sq so if kitch_sq >= life_sq ==> NaN
index = test[test.kitch_sq >= test.life_sq].index
test.iloc[index]["kitch_sq"] = np.NaN
# 6. if  num_room = 0 ==> num_room = NaN
index = test[test.num_room == 0].index 
test.iloc[index]["num_room"] = np.NaN
# 7. 
bad_index = [3174, 7313]
test.iloc[bad_index]["num_room"] = np.NaN
# 8.
index = test[test.max_floor == 0].index
test.iloc[index]["max_floor"] = np.NaN
index = test[test.floor == 0].index
test.iloc[index]["floor"] = np.NaN
# 9. 
index = test[test.floor > test.max_floor].index
test.iloc[index]["max_floor"] = np.NaN

# 10.
index = test.iloc[(test.state >= 5).values].index
test.iloc[index]["state"] = np.NaN


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

In [9]:
train.product_type.value_counts(normalize= True)
test.product_type.value_counts(normalize= True)

Investment       0.655132
OwnerOccupier    0.344868
Name: product_type, dtype: float64

In [10]:
train['month'] = train.timestamp.dt.month
train['day'] = train.timestamp.dt.dayofweek

test['month'] = test.timestamp.dt.month
test['day'] = test.timestamp.dt.dayofweek

In [11]:
# Add some other feature engineering
train['rel_floor'] = train['floor'] / train['max_floor'].astype(float)
train['rel_kitch_sq'] = train['kitch_sq'] / train['full_sq'].astype(float)

test['rel_floor'] = test['floor'] / test['max_floor'].astype(float)
test['rel_kitch_sq'] = test['kitch_sq'] / test['full_sq'].astype(float)

train['room_size'] = train['life_sq'] / train['num_room'].astype(float)
test['room_size'] = test['life_sq'] / test['num_room'].astype(float)


In [12]:
y_train = train["price_doc"]
x_train = train.drop(["id", "timestamp", "price_doc"], axis=1)
x_test = test.drop(["id", "timestamp"], axis=1)


In [13]:
for c in x_train.columns:    
    if x_train[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()  ## Encode target labels with value between 0 and n_classes-1.
        lbl.fit(list(x_train[c].values))  # Fit label encoder.
        x_train[c] = lbl.transform(list(x_train[c].values)) # Fit label encoder and return encoded labels.

for c in x_test.columns:
    if x_test[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(x_test[c].values)) 
        x_test[c] = lbl.transform(list(x_test[c].values))

# First Model

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import mpl_toolkits
%matplotlib inline

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from xgboost.sklearn import XGBRegressor

#No need to run this part of code

In [ ]:
xgb1 = XGBRegressor()
parameters = {'nthread':[5], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [.01, .09], #so called `eta` value
              'max_depth': [7, 9],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [1000]}



In [ ]:
xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 3,
                        n_jobs = 5,
                        verbose=True)


In [ ]:

xgb_grid.fit(x_train, y_train)


Fitting 3 folds for each of 4 candidates, totalling 12 fits


GridSearchCV(cv=3, estimator=XGBRegressor(), n_jobs=5,
             param_grid={'colsample_bytree': [0.7],
                         'learning_rate': [0.01, 0.09], 'max_depth': [7, 9],
                         'min_child_weight': [4], 'n_estimators': [1000],
                         'nthread': [5], 'objective': ['reg:linear'],
                         'silent': [1], 'subsample': [0.7]},
             verbose=True)

In [ ]:
print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

0.6266358150679469
{'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 7, 'min_child_weight': 4, 'n_estimators': 1000, 'nthread': 5, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7}


# First Trial

In [ ]:
#First trial
xgb1 = XGBRegressor(colsample_bytre = 0.7, 
  learning_rate = 0.03,
  max_depth = 5,
  min_child_weight = 4,
  n_estimators = 500,
  nthread = 4, 
  objective = 'reg:linear',
  silent = 1, 
  subsample = 0.7)

In [ ]:
xgb1.fit(x_train,y_train)

XGBRegressor(colsample_bytre=0.7, learning_rate=0.03, max_depth=5,
             min_child_weight=4, n_estimators=500, nthread=4, silent=1,
             subsample=0.7)

In [ ]:
score = xgb1.score(x_train, y_train)  
print("Training score: ", score)

Training score:  0.8289546516644957


In [ ]:
y_predict = xgb1.predict(x_test)
#y_predict = np.expm1(y_predict * 0.99)
output = pd.DataFrame({'id': id_test, 'price_doc': y_predict})
output.head()


,id,price_doc
0,30474,5727185.5
1,30475,8493160.0
2,30476,5508238.5
3,30477,5940526.5
4,30478,5209227.5


In [ ]:
output.to_csv('xgb1.csv', index=False)

# Second trial

In [ ]:
xgb2 = XGBRegressor(colsample_bytree = 0.7, 
                    learning_rate = 0.01, 
                    max_depth = 7,
                    min_child_weight = 4,
                    n_estimators  = 1000,
                    nthread =  5,
                    objective = 'reg:linear',
                    silent = 1,
                    subsample = 0.7)

In [ ]:
xgb2.fit(x_train,y_train)

XGBRegressor(colsample_bytree=0.7, learning_rate=0.01, max_depth=7,
             min_child_weight=4, n_estimators=1000, nthread=5, silent=1,
             subsample=0.7)

In [ ]:

score = xgb2.score(x_train, y_train)  
print("Training score: ", score)


Training score:  0.8712011076776839


In [ ]:
y_predict = xgb2.predict(x_test)
output = pd.DataFrame({'id': id_test, 'price_doc': y_predict})
output.head()

,id,price_doc
0,30474,5495654.5
1,30475,8389220.0
2,30476,5525753.5
3,30477,5843437.0
4,30478,5117540.5


In [ ]:
output.to_csv('xgb2.csv', index=False)

# Neural Network

In [14]:
import keras
import tensorflow as tf
import numpy as np
import pandas as pd

In [19]:
'''

def convert_to_float(x):
    try:
        return np.float(x)
    except:
        return np.nan

x_train = convert_to_float(x_train)
x_test = convert_to_float(x_test)
y_train = convert_to_float(y_train)

'''

In [15]:
# Convert y_train to DataFrame
y_train = pd.DataFrame(y_train)
type(y_train)

pandas.core.frame.DataFrame

In [16]:
# Replace NaN with 0
x_train = x_train.replace(np.nan, 0.0)
x_test = x_test.replace(np.nan, 0.0)
y_train = y_train.replace(np.nan, 0.0)

In [17]:
x_train.head()

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,...,cafe_count_3000_price_500,cafe_count_3000_price_1000,cafe_count_3000_price_1500,cafe_count_3000_price_2500,cafe_count_3000_price_4000,cafe_count_3000_price_high,big_church_count_3000,church_count_3000,mosque_count_3000,leisure_count_3000,sport_count_3000,market_count_3000,green_part_5000,prom_part_5000,office_count_5000,office_sqm_5000,trc_count_5000,trc_sqm_5000,cafe_count_5000,cafe_sum_5000_min_price_avg,cafe_sum_5000_max_price_avg,cafe_avg_price_5000,cafe_count_5000_na_price,cafe_count_5000_price_500,cafe_count_5000_price_1000,cafe_count_5000_price_1500,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,month,day,rel_floor,rel_kitch_sq,room_size
0,43,27.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0,9,6.407578e+06,155572,0.189727,0.000070,9576,5001.0,5,10309,11065.0,5,0,240.0,1,0,7,3,0,0,16,1,0,0,0,0,0,0,0,0,86206,...,21,22,16,3,1,0,2,4,0,0,21,1,13.09,13.31,29,807385,52,4036616,152,708.57,1185.71,947.14,12,39,48,40,9,4,0,13,22,1,0,52,4,8,5,0.0,0.0,0.0
1,34,19.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0,70,9.589337e+06,115352,0.372602,0.049637,6880,3119.0,5,7759,6237.0,8,0,229.0,1,0,6,1,1,1,3,0,0,0,0,0,0,0,0,0,76284,...,11,11,4,2,1,0,1,7,0,6,19,1,10.26,27.47,66,2690465,40,2034942,177,673.81,1148.81,911.31,9,49,65,36,15,3,0,15,29,1,10,66,14,8,1,0.0,0.0,0.0
2,43,29.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0,129,4.808270e+06,101708,0.112560,0.118537,5879,1463.0,4,6207,5580.0,7,0,1183.0,1,0,5,1,0,0,0,1,0,0,0,1,0,0,0,0,101982,...,9,17,9,3,1,0,0,11,0,0,20,6,13.69,21.58,43,1478160,35,1572990,122,702.68,1196.43,949.55,10,29,45,25,10,3,0,11,27,0,4,67,10,8,5,0.0,0.0,0.0
3,89,50.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0,65,1.258354e+07,178473,0.194703,0.069753,13087,6839.0,9,13670,17063.0,10,0,0.0,1,0,17,6,0,0,11,4,0,0,0,0,0,0,0,0,21155,...,5,14,10,3,0,0,1,2,0,0,18,3,14.18,3.89,8,244166,22,942180,61,931.58,1552.63,1242.11,4,7,21,15,11,2,1,4,4,0,0,26,3,9,3,0.0,0.0,0.0
4,77,77.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0,6,8.398461e+06,108171,0.015234,0.037316,5706,3240.0,7,6748,7770.0,9,0,562.0,4,2,25,2,0,0,10,93,0,0,0,1,1,0,0,0,28179,...,266,267,262,149,57,4,70,121,1,40,77,5,8.38,10.92,689,8404624,114,3503058,2283,853.88,1411.45,1132.66,143,566,578,552,319,108,17,135,236,2,91,195,14,9,0,0.0,0.0,0.0


In [18]:
x_train = x_train.astype("float32")
y_train = y_train.astype("float32")

x_test = x_test.astype("float32")

In [19]:
batch_size = 16

model = keras.Sequential([
                                 
                                 keras.layers.Dense(294, activation='relu', input_shape=(294,)),
                                 keras.layers.Dense(294, activation='relu'),
                                 keras.layers.Dense(1, activation='relu')
])

model.compile(optimizer='adam', loss='mean_squared_error')

In [20]:
model.fit(x_train, y_train, epochs=30, batch_size=batch_size)

Epoch 1/30
1905/1905 [==============================] - 6s 3ms/step - loss: nan
Epoch 2/30
1905/1905 [==============================] - 6s 3ms/step - loss: nan
Epoch 3/30
 416/1905 [=====>........................] - ETA: 4s - loss: nan

KeyboardInterrupt: ignored